# 1. Number of Layers vs. Performance

## **Datasets:**
1. German
2. Ionosphere
3. Magic
4. Spambase
5. a8a
6. svmguide3
7. wbc
8. wpbc
9. wdbc

## **Settings:**
1. Full
2. Varying Uniform
3. Varying Gaussian + 0
4. Varying Gaussian + 0.25
5. Varying Gaussian + 0.5
6. Varying Gaussian + 0.75

## **Models:**
1. 2-layer
2. 3-layer
3. 4-layer

In [1]:
import model
import dataloader as dl
import numpy as np
import trainer
import parameters as p
import copy
import seaborn as sns
import matplotlib.pyplot as plt
import torch

torch.manual_seed(p.random_state)
np.random.seed(p.random_state)
model_type = 'jeev_backprop'

In [2]:
datasets = ['german', 'ionosphere', 'spambase', 'magic04', 'a8a']
scenarios = {'varying_gaussian': dl.simulate_varying,
             'varying_gaussian +.25': dl.simulate_varying_25,
             'varying_gaussian +.50': dl.simulate_varying_50,
             'varying_gaussian +.75': dl.simulate_varying_75,
             'varying_uniform': dl.simulate_random_varying,
             'full': dl.simulate_nothing}
layer_settings = [2, 3, 4]

In [3]:
def experiment():
    # initialize the dicts for collecting results
    results = {}
    masks = {}
    occurrences = {}
    losses = {}
    errors = {}

    for dataset in datasets:
        results[dataset] = {}
        masks[dataset] = {}
        losses[dataset] = {}
        errors[dataset] = {}
        
        for scenario in p.scenarios:
            results[dataset][scenario] = {}
            masks[dataset][scenario] = {}
            losses[dataset][scenario] = {}
            errors[dataset][scenario] = {}
            
            for layer in layer_settings:
                results[dataset][scenario][layer] = 0
                masks[dataset][scenario][layer] = []
                losses[dataset][scenario][layer] = []
                errors[dataset][scenario][layer] = []
                

    # run the experiments
    for dataset_name in datasets:
        for scenario in scenarios:
            for layer in layer_settings:
                p.num_layers = layer
                Xpath, ypath = dl.get_path(dataset_name)
                X, y = dl.read_dataset(Xpath, ypath)
                num_features = len(X[0])
                m = p.models[model_type](num_features, p.learning_rate, layer)
                fold_errors, fold_losses, fold_weights, fold_masks, error_lists = trainer.cross_validation(X, y, m, p.folds, p.scenarios[scenario])
                masks[dataset_name][scenario][layer] = fold_masks
                losses[dataset_name][scenario][layer] = fold_losses
                errors[dataset_name][scenario][layer] = error_lists
                results[dataset_name][scenario][layer] = np.mean(fold_errors)
                print(dataset_name, scenario, layer, model_type, np.mean(fold_errors))
            print()
        
    return results, masks, losses, errors

In [ ]:
results, masks, losses, errors = experiment()

german varying_gaussian 2 jeev_backprop 0.38175000000000003
german varying_gaussian 3 jeev_backprop 0.38305000000000006
german varying_gaussian 4 jeev_backprop 0.3822

german varying_gaussian +.25 2 jeev_backprop 0.3843
german varying_gaussian +.25 3 jeev_backprop 0.38384999999999997
german varying_gaussian +.25 4 jeev_backprop 0.37795000000000006

german varying_gaussian +.50 2 jeev_backprop 0.3865
german varying_gaussian +.50 3 jeev_backprop 0.38645000000000007
german varying_gaussian +.50 4 jeev_backprop 0.38395

german varying_gaussian +.75 2 jeev_backprop 0.38575
german varying_gaussian +.75 3 jeev_backprop 0.38875000000000004
german varying_gaussian +.75 4 jeev_backprop 0.3827499999999999

german varying_uniform 2 jeev_backprop 0.3853
german varying_uniform 3 jeev_backprop 0.38550000000000006
german varying_uniform 4 jeev_backprop 0.38089999999999996

german full 2 jeev_backprop 0.33525000000000005
german full 3 jeev_backprop 0.33480000000000004
german full 4 jeev_backprop 0.3316